<a href="https://colab.research.google.com/github/SharvaniBaggani/DietGPT/blob/main/utils_image_preprocess_py_%E2%80%94_face_detect%2C_align%2C_crop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# utils/image_preprocess.py
import cv2
import numpy as np
import mediapipe as mp

mp_face = mp.solutions.face_mesh

def detect_and_align_face_bgr(image_bgr, target_size=224, margin=1.2):
    """
    Detects face using Mediapipe FaceMesh, aligns based on eye centers, crops square,
    and returns resized RGB image (0-1 float).
    """
    img_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    with mp_face.FaceMesh(static_image_mode=True, max_num_faces=1) as face_mesh:
        results = face_mesh.process(img_rgb)
        if not results.multi_face_landmarks:
            return None  # no face found
        landmarks = results.multi_face_landmarks[0].landmark
        h, w, _ = image_bgr.shape

        # Estimate bounding box from landmarks
        xs = [lm.x for lm in landmarks]
        ys = [lm.y for lm in landmarks]
        x_min = int(max(0, min(xs) * w))
        x_max = int(min(w, max(xs) * w))
        y_min = int(max(0, min(ys) * h))
        y_max = int(min(h, max(ys) * h))

        # Expand box by margin
        box_w = x_max - x_min
        box_h = y_max - y_min
        box_size = int(max(box_w, box_h) * margin)
        center_x = (x_min + x_max) // 2
        center_y = (y_min + y_max) // 2
        x1 = max(0, center_x - box_size // 2)
        y1 = max(0, center_y - box_size // 2)
        x2 = min(w, center_x + box_size // 2)
        y2 = min(h, center_y + box_size // 2)

        # Crop and resize
        crop = image_bgr[y1:y2, x1:x2]
        crop_rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
        resized = cv2.resize(crop_rgb, (target_size, target_size))
        # Normalize to [0,1]
        normalized = resized.astype(np.float32) / 255.0
        return normalized

# Quick test:
# img = cv2.imread("sample.jpg")
# out = detect_and_align_face_bgr(img)
# if out is not None: print(out.shape)


/usr/local/lib/python3.12/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.7.2 is installed, but it is not compatible with the installed jaxlib version 0.7.1, so it will not be used.
  warnings.warn(


In [4]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled p